In [86]:
import numpy as np # библиотека для работы с чиселками
import pandas as pd # data processing, работа с CSV файлами
import matplotlib.pyplot as plt # для графики
import seaborn as sns # аналогично

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

Откроем датасет и посмотрим первые 5 его строчек

In [87]:
dataset = pd.read_csv('laptop_price.csv', encoding='ISO-8859-1')
dataset.head(5)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


In [88]:
dataset.describe()

,laptop_ID,Inches,Price_euros
count,1303.000000,1303.000000,1303.000000
mean,660.155794,15.017191,1123.686992
std,381.172104,1.426304,699.009043
min,1.000000,10.100000,174.000000
25%,331.500000,14.000000,599.000000
50%,659.000000,15.600000,977.000000
75%,990.500000,15.600000,1487.880000
max,1320.000000,18.400000,6099.000000


Т.к. необходимо решить задачу классификации, то разделим цены на 3 категории: низкие, средние и высокие

Для задачи регрессии будем предсказывать цену ноутбука по остальным характеристикам

In [89]:
dataset['Price_category'] = pd.qcut(dataset['Price_euros'], q=3, labels=['Low', 'Medium', 'High'])


Подготовим данные

In [90]:
# Удаление ненужных столбцов
dataset = dataset.drop(columns=['laptop_ID'])
new_dataset = dataset.copy()

# Кодирование категориальных признаков
categorical_columns = ['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight']
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    new_dataset[column] = le.fit_transform(new_dataset[column])
    label_encoders[column] = le  # Сохраняем энкодеры, если понадобится обратное преобразование


# Разделение на признаки и целевую переменную
X = new_dataset.drop(columns=['Price_euros', 'Price_category'])
y_classification = new_dataset['Price_category']
y_regression = new_dataset['Price_euros']

# Масштабирование числовых признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


Разделим датасет на обучающую и тестовую выборку

In [91]:
X_train, X_test, y_train_classification, y_test_classification = train_test_split(X_scaled, y_classification, test_size=0.2, random_state=42)

In [92]:
X_train.shape

(1042, 11)

In [93]:
y_train_classification


10         Low
147        Low
1287    Medium
767     Medium
816       High
         ...  
1095    Medium
1130       Low
1294       Low
860     Medium
1126       Low
Name: Price_category, Length: 1042, dtype: category
Categories (3, object): ['Low' < 'Medium' < 'High']

Обучим модель Градиентного бустинга

In [94]:
model_classification = GradientBoostingClassifier(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=3, 
    random_state=42
)

model_classification.fit(X_train, y_train_classification)

GradientBoostingClassifier(random_state=42)

Оценим работу классификатора

In [95]:
train_predict_classification = model_classification.predict(X_train)
test_predict_classification = model_classification.predict(X_test)

In [96]:
# Расчет метрик
train_f1_classification = f1_score(y_train_classification, train_predict_classification, average='micro')
train_accuracy_classification = accuracy_score(y_train_classification, train_predict_classification)
train_precision_classification = precision_score(y_train_classification, train_predict_classification, average='micro')
train_recall_classification = recall_score(y_train_classification, train_predict_classification, average='micro')

test_f1_classification = f1_score(y_test_classification, test_predict_classification, average='micro')
test_accuracy_classification = accuracy_score(y_test_classification, test_predict_classification)
test_precision_classification = precision_score(y_test_classification, test_predict_classification, average='micro')
test_recall_classification = recall_score(y_test_classification, test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', train_f1_classification, train_accuracy_classification, train_precision_classification, train_recall_classification)
print('Test', test_f1_classification, test_accuracy_classification, test_precision_classification, test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.9520153550863724 0.9520153550863724 0.9520153550863724 0.9520153550863724
Test 0.8352490421455939 0.8352490421455939 0.8352490421455939 0.8352490421455939


### Теперь посмотрим работу алгоритма Градиентного бустинга для задачи регрессии

Разделим датасет на обучающую и тестовую выборку

In [97]:
X_train, X_test, y_train_regression, y_test_regression = train_test_split(X_scaled, y_regression, test_size=0.2, random_state=42)

Обучим модель

In [98]:
model_regression = GradientBoostingRegressor(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=3, 
    random_state=42
)

model_regression.fit(X_train, y_train_regression)

GradientBoostingRegressor(random_state=42)

Оценим работу

In [99]:
train_predict_regression = model_regression.predict(X_train)
test_predict_regression = model_regression.predict(X_test)

In [100]:
# Расчет метрик
train_mse_regression = mean_squared_error(y_train_regression, train_predict_regression)
train_mae_regression = mean_absolute_error(y_train_regression, train_predict_regression)

test_mse_regression = mean_squared_error(y_test_regression, test_predict_regression)
test_mae_regression = mean_absolute_error(y_test_regression, test_predict_regression)



print('Метрика:          MSE            MAE')
print('Train', train_mse_regression, train_mae_regression)
print('Test', test_mse_regression, test_mae_regression)

Метрика:          MSE            MAE
Train 38774.78788351394 141.86327441228985
Test 65722.88793048693 171.4187400362743


### Вывод:
В задаче классификации метрики на тренировочных данных показывают высокую производительность модели с F1-мерой 0.95, что свидетельствует о качественном обучении, а метрики на тестовых данных остаются на достойном уровне (F1 = 0.83), что говорит о хорошем обобщении модели.
В задаче регрессии значения ошибок на тренировочных данных (MSE = 38774.79, MAE = 141.86) показывают точность модели при обучении, а ошибки на тестовых данных (MSE = 65722.89, MAE = 171.42) немного выше, что ожидаемо, но не свидетельствует о сильной переобученности.

Модель градиентного бустинга демонстрирует высокую точность на тренировочных данных, удовлетворительное обобщение на тестовом наборе и небольшое расхождение между тренировочными и тестовыми метриками, что говорит о хорошем балансе между переобучением и недообучением.

# Улучшение бейзлайна

Для улучшения бейзлайна модели в задачах классификации и регрессии предлагаю следующие решения:

Удалить столбцы с высокой кореляцией, закодировать категориальные признаки более информативно (используя One-Hot Encoding), а также использовать метод GridSearchCV для поиска лучших параметров.

Для начала посмотрим матрицу кореляций для данного датасета

In [101]:
X.corr()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
Company,1.000000,0.067234,-0.007718,-0.085784,0.094772,0.044593,-0.008399,0.048086,0.026537,0.134290,-0.159906
Product,0.067234,1.000000,0.065594,-0.218974,0.176393,0.143855,0.018305,0.088667,0.094795,0.120562,-0.204379
TypeName,-0.007718,0.065594,1.000000,-0.077428,-0.175618,-0.128174,0.213988,0.019091,-0.204774,0.085223,-0.211832
Inches,-0.085784,-0.218974,-0.077428,1.000000,-0.247841,0.153041,-0.149706,-0.193382,0.215710,0.034528,0.866304
ScreenResolution,0.094772,0.176393,-0.175618,-0.247841,1.000000,0.232751,0.019858,0.059042,0.160816,0.070360,-0.218135
Cpu,0.044593,0.143855,-0.128174,0.153041,0.232751,1.000000,-0.077176,-0.013264,0.490761,0.125374,0.139861
Ram,-0.008399,0.018305,0.213988,-0.149706,0.019858,-0.077176,1.000000,-0.248580,-0.138212,0.008627,-0.174577
Memory,0.048086,0.088667,0.019091,-0.193382,0.059042,-0.013264,-0.248580,1.000000,-0.020784,0.035574,-0.218299
Gpu,0.026537,0.094795,-0.204774,0.215710,0.160816,0.490761,-0.138212,-0.020784,1.000000,0.103853,0.274961
OpSys,0.134290,0.120562,0.085223,0.034528,0.070360,0.125374,0.008627,0.035574,0.103853,1.000000,-0.031506


Можно заметить, что у Weight и Inches коэффициент кореляции 0.86, что достаточно много, значит уберем один из этих столбцов (пусть это будет Weight)

In [102]:
dataset = dataset.drop(columns=['Weight'])

In [103]:
dataset

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Price_euros,Price_category
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1339.69,High
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,898.94,Medium
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,575.00,Low
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,2537.45,High
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1803.60,High
...,...,...,...,...,...,...,...,...,...,...,...,...
1298,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,638.00,Low
1299,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,Windows 10,1499.00,High
1300,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,229.00,Low
1301,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,764.00,Medium


In [104]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [111]:
# Создание пайплайна для классификации
categorical_features = ['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys']
numerical_features = ['Inches']

X = dataset.drop(columns=['Price_euros', 'Price_category'])

new_X_train, new_X_test, y_train_regression, y_test_regression = train_test_split(X, y_regression, test_size=0.2, random_state=42)
_, _, y_train_classification, y_test_classification = train_test_split(X, y_classification, test_size=0.2, random_state=42)


# Пайплайн для классификации
classification_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier())
])

# Пайплайн для регрессии
regression_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

param_grid_classification = {
    'classifier__n_estimators': [50, 100, 150],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__max_depth': [3, 5, 7],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2]
}

param_grid_regression = {
    'regressor__n_estimators': [50, 100, 150],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__max_depth': [3, 5, 7],
    'regressor__min_samples_split': [2, 5],
    'regressor__min_samples_leaf': [1, 2]
}

grid_search_classification = GridSearchCV(classification_pipeline, param_grid_classification)
grid_search_classification.fit(new_X_train, y_train_classification)

grid_search_regression = GridSearchCV(regression_pipeline, param_grid_regression)
grid_search_regression.fit(new_X_train, y_train_regression)

print("Best parameters for classification:", grid_search_classification.best_params_)
print("Best parameters for regression:", grid_search_regression.best_params_)


Best parameters for classification: {'classifier__learning_rate': 0.2, 'classifier__max_depth': 3, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 150}
Best parameters for regression: {'regressor__learning_rate': 0.2, 'regressor__max_depth': 7, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 150}


In [ ]:
preprocessor

ColumnTransformer(transformers=[('num', StandardScaler(), ['Inches']),
                                ('cat',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['Company', 'Product', 'TypeName',
                                  'ScreenResolution', 'Cpu', 'Ram', 'Memory',
                                  'Gpu', 'OpSys'])])

In [112]:
preprocessor.fit(new_X_train)

X_train_preprocessed = preprocessor.transform(new_X_train)
X_test_preprocessed = preprocessor.transform(new_X_test)

Обучим модели и оценим их работу

In [113]:
new_model_classification = GradientBoostingClassifier(learning_rate=0.2, max_depth=3, min_samples_leaf=2, min_samples_split=2, n_estimators=150)

new_model_classification.fit(X_train_preprocessed, y_train_classification)

GradientBoostingClassifier(learning_rate=0.2, min_samples_leaf=2,
                           n_estimators=150)

In [114]:
new_model_regression = GradientBoostingRegressor(learning_rate=0.2, max_depth=7, min_samples_leaf=1, min_samples_split=2, n_estimators=150)

new_model_regression.fit(X_train_preprocessed, y_train_regression)

GradientBoostingRegressor(learning_rate=0.2, max_depth=7, n_estimators=150)

In [115]:
new_train_predict_classification = new_model_classification.predict(X_train_preprocessed)
new_test_predict_classification = new_model_classification.predict(X_test_preprocessed)

In [116]:
# Расчет метрик
new_train_f1_classification = f1_score(y_train_classification, new_train_predict_classification, average='micro')
new_train_accuracy_classification = accuracy_score(y_train_classification, new_train_predict_classification)
new_train_precision_classification = precision_score(y_train_classification, new_train_predict_classification, average='micro')
new_train_recall_classification = recall_score(y_train_classification, new_train_predict_classification, average='micro')

new_test_f1_classification = f1_score(y_test_classification, new_test_predict_classification, average='micro')
new_test_accuracy_classification = accuracy_score(y_test_classification, new_test_predict_classification)
new_test_precision_classification = precision_score(y_test_classification, new_test_predict_classification, average='micro')
new_test_recall_classification = recall_score(y_test_classification, new_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', new_train_f1_classification, new_train_accuracy_classification, new_train_precision_classification, new_train_recall_classification)
print('Test', new_test_f1_classification, new_test_accuracy_classification, new_test_precision_classification, new_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.9712092130518234 0.9712092130518234 0.9712092130518234 0.9712092130518234
Test 0.8122605363984674 0.8122605363984674 0.8122605363984674 0.8122605363984674


In [117]:
new_train_predict_regression = new_model_regression.predict(X_train_preprocessed)
new_test_predict_regression = new_model_regression.predict(X_test_preprocessed)

In [118]:
# Расчет метрик
new_train_mse_regression = mean_squared_error(y_train_regression, new_train_predict_regression)
new_train_mae_regression = mean_absolute_error(y_train_regression, new_train_predict_regression)

new_test_mse_regression = mean_squared_error(y_test_regression, new_test_predict_regression)
new_test_mae_regression = mean_absolute_error(y_test_regression, new_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', new_train_mse_regression, new_train_mae_regression)
print('Test', new_test_mse_regression, new_test_mae_regression)

Метрика:          MSE            MAE
Train 4895.6252352834 56.101123398873156
Test 103612.93797765087 180.8199618520582


### Вывод:
До оптимизации модель демонстрировала хорошую производительность на тренировочной выборке по метрикам F1, Accuracy, Precision и Recall (0.95). Однако на тестовой выборке наблюдалась некоторая потеря качества (F1 = 0.835). Метрики MSE и MAE были на приемлемом уровне, но указывали на более высокую ошибку на тестовой выборке (MSE = 65722.88, MAE = 171.42).

После оптимизации точность модели на тренировочной выборке значительно выросла (F1 = 0.97), что говорит о лучшей адаптации модели к обучающим данным, однако качество на тестовой выборке снизилось (F1 = 0.812), что может свидетельствовать о переобучении или необходимости более тщательной настройки гиперпараметров. Метрики MSE и MAE заметно улучшились на тренировочной выборке (MSE = 4895.63, MAE = 56.10), но ошибка на тестовой выборке увеличилась (MSE = 103612.94, MAE = 180.82).

Общий вывод: оптимизация модели улучшила её способность обобщать данные на тренировочной выборке, но снизила эффективность на тестовой выборке. Это указывает на необходимость дальнейшего изучения гиперпараметров или использования дополнительных методов регуляризации для достижения лучшего баланса между переобучением и недообучением.

### Имплементация алгоритма

In [119]:
from collections import Counter
from sklearn.base import BaseEstimator

In [120]:
class GRADIENTBOOSTINGCLASSIFIER(BaseEstimator):
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, random_state=None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.trees = []
        self.classes_ = np.unique(y)
        self.init_prediction = np.log(np.mean(y) / (1 - np.mean(y)))
        predictions = np.full(len(y), self.init_prediction)

        for _ in range(self.n_estimators):
            # Compute the pseudo residuals
            residuals = y - 1 / (1 + np.exp(-predictions))

            # Fit a weak learner (decision tree) to the residuals
            from sklearn.tree import DecisionTreeRegressor
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(X, residuals)

            # Update the predictions
            update = self.learning_rate * tree.predict(X)
            predictions += update

            # Store the tree
            self.trees.append(tree)

    def predict_proba(self, X):
        # Compute the final predictions
        predictions = np.full(X.shape[0], self.init_prediction)
        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(X)
        # Convert logits to probabilities
        probabilities = 1 / (1 + np.exp(-predictions))
        return np.column_stack([1 - probabilities, probabilities])

    def predict(self, X):
        # Convert probabilities to class labels
        probabilities = self.predict_proba(X)[:, 1]
        return (probabilities >= 0.5).astype(int)


class GRADIENTBOOSTINGREGRESSOR(BaseEstimator):
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, random_state=None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.trees = []
        self.init_prediction = np.mean(y)  # Начальное предсказание - среднее значение
        predictions = np.full(len(y), self.init_prediction)

        for _ in range(self.n_estimators):
            # Вычисление псевдоостатков
            residuals = y - predictions

            # Обучение дерева на остатки
            from sklearn.tree import DecisionTreeRegressor
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(X, residuals)

            # Обновление предсказаний
            update = self.learning_rate * tree.predict(X)
            predictions += update

            # Сохранение дерева
            self.trees.append(tree)

    def predict(self, X):
        # Предсказание на основе всех деревьев
        predictions = np.full(X.shape[0], self.init_prediction)
        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(X)
        return predictions

In [121]:
from sklearn.preprocessing import LabelEncoder

# Кодирование целевой переменной
label_encoder = LabelEncoder()
y_train_classification_encoded = label_encoder.fit_transform(y_train_classification)
y_test_classification_encoded = label_encoder.fit_transform(y_test_classification)

In [126]:
# Применение собственной реализации для классификации
gb_classifier = GRADIENTBOOSTINGCLASSIFIER(n_estimators=100, learning_rate=0.2, max_depth=15, random_state=42)
gb_classifier.fit(X_train, y_train_classification_encoded)

my_train_predict_classification = gb_classifier.predict(X_train)
my_test_predict_classification = gb_classifier.predict(X_test)

In [127]:
# Расчет метрик
my_train_f1_classification = f1_score(y_train_classification_encoded, my_train_predict_classification, average='micro')
my_train_accuracy_classification = accuracy_score(y_train_classification_encoded, my_train_predict_classification)
my_train_precision_classification = precision_score(y_train_classification_encoded, my_train_predict_classification, average='micro')
my_train_recall_classification = recall_score(y_train_classification_encoded, my_train_predict_classification, average='micro')

my_test_f1_classification = f1_score(y_test_classification_encoded, my_test_predict_classification, average='micro')
my_test_accuracy_classification = accuracy_score(y_test_classification_encoded, my_test_predict_classification)
my_test_precision_classification = precision_score(y_test_classification_encoded, my_test_predict_classification, average='micro')
my_test_recall_classification = recall_score(y_test_classification_encoded, my_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', my_train_f1_classification, my_train_accuracy_classification, my_train_precision_classification, my_train_recall_classification)
print('Test', my_test_f1_classification, my_test_accuracy_classification, my_test_precision_classification, my_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.6765834932821497 0.6765834932821497 0.6765834932821497 0.6765834932821497
Test 0.524904214559387 0.524904214559387 0.524904214559387 0.524904214559387


In [128]:

# Применение собственной реализации для регрессии
gb_regressor = GRADIENTBOOSTINGREGRESSOR(n_estimators=100, learning_rate=0.2, max_depth=10, random_state=42)
gb_regressor.fit(X_train, y_train_regression)

my_train_predict_regression = gb_regressor.predict(X_train)
my_test_predict_regression = gb_regressor.predict(X_test)

In [129]:
# Расчет метрик
my_train_mse_regression = mean_squared_error(y_train_regression, my_train_predict_regression)
my_train_mae_regression = mean_absolute_error(y_train_regression, my_train_predict_regression)

my_test_mse_regression = mean_squared_error(y_test_regression, my_test_predict_regression)
my_test_mae_regression = mean_absolute_error(y_test_regression, my_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', my_train_mse_regression, my_train_mae_regression)
print('Test', my_test_mse_regression, my_test_mae_regression)

Метрика:          MSE            MAE
Train 107.17682111004393 1.9540420045575528
Test 75297.17555121098 164.94003716749944


Посмотрим работу алгоритма на улучшенном датасете

In [131]:
# Применение собственной реализации для классификации
gb_classifier = GRADIENTBOOSTINGCLASSIFIER(n_estimators=100, learning_rate=0.2, max_depth=15, random_state=42)
gb_classifier.fit(X_train_preprocessed, y_train_classification_encoded)

new_my_train_predict_classification = gb_classifier.predict(X_train_preprocessed)
new_my_test_predict_classification = gb_classifier.predict(X_test_preprocessed)

In [132]:
# Расчет метрик
new_my_train_f1_classification = f1_score(y_train_classification_encoded, new_my_train_predict_classification, average='micro')
new_my_train_accuracy_classification = accuracy_score(y_train_classification_encoded, new_my_train_predict_classification)
new_my_train_precision_classification = precision_score(y_train_classification_encoded, new_my_train_predict_classification, average='micro')
new_my_train_recall_classification = recall_score(y_train_classification_encoded, new_my_train_predict_classification, average='micro')

new_my_test_f1_classification = f1_score(y_test_classification_encoded, new_my_test_predict_classification, average='micro')
new_my_test_accuracy_classification = accuracy_score(y_test_classification_encoded, new_my_test_predict_classification)
new_my_test_precision_classification = precision_score(y_test_classification_encoded, new_my_test_predict_classification, average='micro')
new_my_test_recall_classification = recall_score(y_test_classification_encoded, new_my_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', new_my_train_f1_classification, new_my_train_accuracy_classification, new_my_train_precision_classification, new_my_train_recall_classification)
print('Test', new_my_test_f1_classification, new_my_test_accuracy_classification, new_my_test_precision_classification, new_my_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.6669865642994242 0.6669865642994242 0.6669865642994242 0.6669865642994242
Test 0.5440613026819924 0.5440613026819924 0.5440613026819924 0.5440613026819924


In [133]:

# Применение собственной реализации для регрессии
gb_regressor = GRADIENTBOOSTINGREGRESSOR(n_estimators=100, learning_rate=0.2, max_depth=10, random_state=42)
gb_regressor.fit(X_train_preprocessed, y_train_regression)

new_my_train_predict_regression = gb_regressor.predict(X_train_preprocessed)
new_my_test_predict_regression = gb_regressor.predict(X_test_preprocessed)

In [134]:
# Расчет метрик
new_my_train_mse_regression = mean_squared_error(y_train_regression, new_my_train_predict_regression)
new_my_train_mae_regression = mean_absolute_error(y_train_regression, new_my_train_predict_regression)

new_my_test_mse_regression = mean_squared_error(y_test_regression, new_my_test_predict_regression)
new_my_test_mae_regression = mean_absolute_error(y_test_regression, new_my_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', new_my_train_mse_regression, new_my_train_mae_regression)
print('Test', new_my_test_mse_regression, new_my_test_mae_regression)

Метрика:          MSE            MAE
Train 1829.6424315038128 33.966690813311416
Test 117539.1537572526 192.9072760437604


##### Сравнение стандартной и собственной реализации Градиентного бустинга до улучшений:
До оптимизации модель показала низкую производительность на тренировочной выборке (F1 = 0.676) и ещё более низкие результаты на тестовой (F1 = 0.525). Ошибки (MSE = 75297.18, MAE = 164.94) на тестовой выборке оказались выше, чем у библиотечной реализации, что указывает на меньшую способность модели к обобщению.

##### Сравнение стандартной и собственной реализации Градиентного бустинга после улучшения бейзлайна:
После оптимизации показатели на тренировочной выборке остались примерно на том же уровне (F1 = 0.667), а на тестовой немного улучшились (F1 = 0.544). Метрики ошибок (MSE и MAE) улучшились на тренировочной выборке (MSE = 1829.64, MAE = 33.97), но остались высокими на тестовой выборке (MSE = 117539.15, MAE = 192.91).


### Вывод:
Библиотечная реализация изначально показала значительно лучшие результаты, однако после оптимизации начались признаки переобучения.
Собственная реализация после оптимизации показала небольшое улучшение качества на тестовой выборке, но осталась далеко позади по сравнению с библиотечной. Хотя можно поиграть с разными параметриами и улучшить результат собственной реализации, но он все равно будет отставать от библиотечного варианта.